## Demonstration of a Simple Cosine Similarity Content-Based Recommendation System
In this file, the demonstration of the cosine similarity content-based recommender model recommending news is presented. 
It also includes the evaluation of the recommender model using the metrics *MAP@K and NDCG@K*.

In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from utils.process_data import user_item_interaction_scores
from parquet_data_reader import ParquetDataReader
from models.content_based.content_based import CosineSimilarityContentBased

import polars as pl
import numpy as np
parquet_reader = ParquetDataReader()

### Reading Data and Preprocessing

In [2]:
train_behavior_df = parquet_reader.read_data("../../data/train/behaviors.parquet")
embeddings_df = parquet_reader.read_data("../../data/document_vector.parquet")
article_df = parquet_reader.read_data("../../data/articles.parquet")
test_behavior_df = parquet_reader.read_data("../../data/validation/behaviors.parquet")
processed_behavior_df = user_item_interaction_scores(train_behavior_df, article=article_df)
train_history_df =  pl.read_parquet("../../data/train/history.parquet")

### Recommendations

In [3]:
content_based = CosineSimilarityContentBased(behavior_data=processed_behavior_df, item_data=embeddings_df)

recommendations = content_based.recommend(14241, 5)
score = content_based.score(14241, 6044108)

print("Recommendations for user 14241")
print(recommendations)
print("Score for user 14241 and item 6044108")
print(score)

c:\Users\magnu\NewDesk\An.sys\TDT4215\recommender_system\models\content_based\content_based.py:153: RuntimeWarning: invalid value encountered in scalar divide
  similarity = np.dot(user_vector, item_vector) / (


Recommendations for user 14241
[7160857, 7789111, 6044108, 4925298, 7124272]
Score for user 14241 and item 6044108
0.9761224341910408


### Evaluation of the Cosine Similarity Content-Based Recommendation System

In [4]:
results = content_based.evaluate_recommender(k=5,n_jobs=4,user_sample=15)

In [5]:
print("Results")
results

Results


{'MAP@K': np.float64(0.0), 'NDCG@K': np.float64(0.0)}

### Evaluation for Comparison

In [6]:
#TODO

In [7]:
users_df = train_history_df["user_id"].unique()
users_df


user_id
u32
10068
10200
10201
10623
10701
…
2590015
2590054
2590471


### Aggregate Diversity Evaluation
Calculates the aggrigate diversity of the recommender model recommendations, and appends the result to the `/evaluation_summary/model_overview_diversity.csv`-file. 

In [8]:
from utils.evaluation import aggregate_diversity
from utils.evaluation import append_aggregate_diversity

diversity = aggregate_diversity(content_based, article_df, users_df=users_df, user_sample=15)

print("Diversity")
print(diversity)

append_aggregate_diversity(diversity, "content based")

c:\Users\magnu\NewDesk\An.sys\TDT4215\recommender_system\models\content_based\content_based.py:153: RuntimeWarning: invalid value encountered in scalar divide
  similarity = np.dot(user_vector, item_vector) / (


Diversity
0.0026521361751374287


### Gini Coefficient
Prints the gini coefficient for the recommendations